# Módulos

In [128]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV # Dividir set de datos # Busca en una cuadricula la mejor combinacion de hiperparametros
from sklearn.ensemble import RandomForestRegressor # Algoritmo de ML que usaremos

import warnings
warnings.filterwarnings('ignore')

# Lectura de datos

In [129]:
df = pd.read_csv('laptop_prices.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Data columns (total 23 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Company               1275 non-null   object 
 1   Product               1275 non-null   object 
 2   TypeName              1275 non-null   object 
 3   Inches                1275 non-null   float64
 4   Ram                   1275 non-null   int64  
 5   OS                    1275 non-null   object 
 6   Weight                1275 non-null   float64
 7   Price_euros           1275 non-null   float64
 8   Screen                1275 non-null   object 
 9   ScreenW               1275 non-null   int64  
 10  ScreenH               1275 non-null   int64  
 11  Touchscreen           1275 non-null   object 
 12  IPSpanel              1275 non-null   object 
 13  RetinaDisplay         1275 non-null   object 
 14  CPU_company           1275 non-null   object 
 15  CPU_freq             

# Codificación One-Hot


¿Cómo funciona la codificación one-hot?
<br></br>
- Para cada variable categórica, crea nuevas columnas binarias (0 o 1) que representan cada categoría.

In [130]:
#Para determinar qué columnas añadir a get_dummies, inspecciona tu df y considera qué columnas contienen variables categóricas que te gustaría convertir a variables numéricas.

df = pd.get_dummies(df, columns = ["Company","Product","TypeName","OS","Screen","Touchscreen","IPSpanel","RetinaDisplay","CPU_company","CPU_model","PrimaryStorageType","SecondaryStorageType","GPU_company","GPU_model"] ,drop_first=True)


In [131]:
df.head(2)

,Inches,Ram,Weight,Price_euros,ScreenW,ScreenH,CPU_freq,PrimaryStorage,SecondaryStorage,Company_Apple,...,GPU_model_Radeon R7 M440,GPU_model_Radeon R7 M445,GPU_model_Radeon R7 M460,GPU_model_Radeon R7 M465,GPU_model_Radeon R9 M385,GPU_model_Radeon RX 540,GPU_model_Radeon RX 550,GPU_model_Radeon RX 560,GPU_model_Radeon RX 580,GPU_model_UHD Graphics 620
0,13.3,8,1.37,1339.69,2560,1600,2.3,128,0,True,...,False,False,False,False,False,False,False,False,False,False
1,13.3,8,1.34,898.94,1440,900,1.8,128,0,True,...,False,False,False,False,False,False,False,False,False,False


Ejemplo:
- Si tu DataFrame df contiene una columna llamada "Color" con los valores "Rojo", "Verde" y "Azul", podrías usar get_dummies para crear tres nuevas columnas: "Color_Rojo", "Color_Verde" y "Color_Azul". Cada fila tendría un 1 en la columna correspondiente a su color y 0 en las demás.

In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1275 entries, 0 to 1274
Columns: 875 entries, Inches to GPU_model_UHD Graphics 620
dtypes: bool(866), float64(4), int64(5)
memory usage: 1.1 MB


- Se han creado 17 columnas nuevas de la columna 'Company'
- Se han creado 2 columnas nuevas de la columna 'CPU_company'

# Separación de Datos


- Este código es un paso fundamental en la preparación de datos para Machine Learning. Está separando las características de entrada de la variable objetivo.

In [133]:
y = df["Price_euros"]
x = df.drop("Price_euros", axis = 1)

- Train Test Split divide los datos

- Test Size Indica el porcentaje que se asigna al conjunto de prueba. El resto se usa para el train

- Random State establece una semilla para el generador de números aleatorios. Esto garantiza que la división sea reproducible. Si ejecutas el código nuevamente, obtendrás la misma división.

In [134]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.30, random_state=42)
# Si tienes un conjunto de datos muy grande, podrías usar un test_size más pequeño.

x - Se divide en X_train (70% de los datos);  y X_test (30% de los datos)

y - Se divide en Y_train (70% de los datos);  y Y_test (30% de los datos)

- - -

# Creación del Modelo

**RandomForestRegressor** ; Crea una instancia del modelo. Se puede ajustar los hiperparametros dentro del parentesis

**fit** ; Entrena el modelo 'rf' usando datos de x_train (caracteristicas) e 'y_train' (variable objetivo)

**Score** ; Evalua el rendimiento usando los datos de prueba x_test e y_test

In [135]:
rf = RandomForestRegressor(max_depth=15, min_samples_leaf=1, min_samples_split=5, n_estimators=200)
model = rf.fit(x_train, y_train)
model.score(x_test, y_test)

0.8537948533553863

# Probamos modelo

con y.iloc podemos ver el precio original de un laptop en los datos reales ya que con esta funcion podemos acceder a la información de una columna concreta

In [136]:
y.iloc[200]

# y ; contiene la columna 'Price euros' del dataframe original
# .iloc[] se usa para seleccionar datos por su posicion numerica en un objeto de pandas
# [200]; indica que se quiere acceder al elemento en posición 200
# Conclusión : dará el precio en euros del portátil en la fila 200 del DataFrame original.

1845.0

Con estas líneas de código buscamos la mejor combinación de hiperparametros

In [138]:
model.predict([x.iloc[200]])

array([1940.85442493])

In [139]:
model.predict([x.iloc[155]])

array([749.1717117])

Con estas líneas de código buscamos la mejor combinación de hiperparametros

In [139]:
"""
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]

grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5)
grid_search.fit(x_train, y_train)

print(grid_search.best_params_)
"""

In [146]:
def predecir_precio():
  numero_dato = int(input("Introduce el número de dato: "))
  numero_real = y.iloc[numero_dato]
  print(f"El precio real del dato {numero_dato} es: {numero_real}")
  prediccion = model.predict([x.iloc[numero_dato]])
  print(f"La predicción del precio para el dato {numero_dato} es: {prediccion[0]}")

# Ejemplo de uso
predecir_precio()

Introduce el número de dato: 25
El precio real del dato 25 es: 418.64
La predicción del precio para el dato 25 es: 468.06796016604414
